In [1]:
def facility_path(instance):
    return 'documents/{}'.format(instance)

In [2]:
facility_path(12)

'documents/12'

In [ ]:
import docx
import pandas as pd
from os import path, listdir

In [ ]:
articles = pd.read_csv(r'jurnal_papers.csv')

In [ ]:
def read_docx(folder_path, folder_name):
    f_Text = []
    fText = ''
    try:
        for name in listdir(folder_path):
            if ((folder_name)== name[:-5]):
                filePath = path.join(folder_path, name)
                print('filePath is:{}'.format(filePath))
                doc = docx.Document(filePath)
                for line in doc.paragraphs:
                    f_Text.append(line.text)
            else:
                continue
    except:
        print("reading {} file failed, no such directory".format(folder_path))

    fText = '\n'.join(f_Text)
    return fText


In [ ]:
for i, a_id in enumerate(articles['folder']):
    link = r'D:\MWT\master_thesis\GCSAR\accepted\{}'.format(a_id)
    filetext = read_docx(link,a_id)
    articles.artc_txt[i] = filetext

filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-1\14-1.docx


C:\Users\User\AppData\Local\Temp/ipykernel_11064/3217126970.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles.artc_txt[i] = filetext
C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-3\14-3.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-4\14-4.docx
reading D:\MWT\master_thesis\GCSAR\accepted\14-5 file failed, no such directory
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-6\14-6.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-8\14-8.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-9\14-9.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-10\14-10.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-11\14-11.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-12\14-12.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-13\14-13.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-14\14-14.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-15\14-15.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-16\14-16.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-17\14-17.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-18\14-18.docx
filePath is:D:\MWT\m

filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-179\17-179.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-181\17-181.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-184\17-184.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-185\17-185.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-187\17-187.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-190\17-190.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-191\17-191.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-192\17-192.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-194\17-194.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-195\17-195.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-197\17-197.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-198\17-198.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-201\17-201.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-202\17-202.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-206\17-206.

filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-366\18-366.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-367\18-367.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-368\18-368.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-371\18-371.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-373\18-373.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-374\18-374.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-375\18-375.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-376\18-376.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-377\18-377.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-378\18-378.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-380\18-380.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-381\18-381.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-382\18-382.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-383\18-383.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-384\18-384.

filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-538\19-538.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-542\19-542.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-547\19-547.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-548\19-548.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-549\19-549.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-550\19-550.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-551\19-551.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-552\19-552.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-553\19-553.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-554\19-554.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-555\19-555.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-556\19-556.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-557\19-557.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-558\19-558.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-559\19-559.

filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-693\20-693.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-694\20-694.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-696\20-696.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-697\20-697.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-698\20-698.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-699\20-699.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-700\20-700.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-701\20-701.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-702\20-702.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-703\20-703.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-706\20-706.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-708\20-708.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-709\20-709.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-710\20-710.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-711\20-711.

In [ ]:
# articles.describe()
articles.head()

In [ ]:
import re
import string
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.utils.dediac import dediac_ar
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from farasa.stemmer import FarasaStemmer

In [ ]:
def remove_numbers_punctuations (text):
    clean_text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', ' ', str(text))
    clean_text = re.sub(r'\w*\d\w*', ' ', str(clean_text))
    clean_text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', clean_text)
    clean_text = re.sub(r'،|؛', ' ', str(clean_text))
    return clean_text

In [ ]:
def tokenize(text):
    tokens = simple_word_tokenize(text)
    return tokens

In [ ]:
def remove_stop_words(tokens):
    filtered_text = [t for t in tokens if not t in stopwords.words("arabic")]
#     filtered_text = [t for t in filtered_text if not t in stopwords.words("english")]
    return filtered_text

In [ ]:
def remove_stop_words2(tokens):
    filtered_text = [t for t in tokens if not t in stopwords.words("english")]
    return filtered_text

In [ ]:
def ar_stemmer(text):
    stemmer = FarasaStemmer()
    stemmed_text = stemmer.stem(text)
    return stemmed_text


In [ ]:
def vectorizer(filtered_text):
    cv = CountVectorizer()
    transformed_text = cv.fit_transform(filtered_text)
    vector_table = pd.DataFrame(transformed_text.toarray(), columns=cv.get_feature_names())
    return vector_table

In [ ]:
def word_counter(word_list):
    counts = Counter(word_list)
    # Reverse the key/values in the dictionary for sorting
    word_counts = list(zip(counts.values(), counts.keys()))

    # Sort the list by count
    word_counts = sorted(word_counts, reverse=True)
    return word_counts

In [ ]:
def remove_single_double_char(text):
    no_singl_char =re.sub(r'[×©±≤≥]', ' ', str(text))
    no_singl_char =re.sub(r'\b\w{1,2}\b', ' ', str(no_singl_char))
#     simpols = '× , © , o , ≤ , ≥'
    return no_singl_char

In [ ]:
def only_ar_text(text):
    ar_text = re.sub(r"[A-Z]['\w]*|[a-z]['\w]*", ' ', str(text))
    return ar_text

In [ ]:
articles['clean_txt'] = articles.artc_txt.map(remove_numbers_punctuations)
# .map(tokenize).map(remove_stop_words)

In [ ]:
articles['clean_txt'] = articles['clean_txt'].map(tokenize)

In [ ]:
articles['clean_txt'] = articles['clean_txt'].map(remove_stop_words)

In [ ]:
articles.clean_txt[0:100]

In [ ]:
for i,txt in enumerate(articles.clean2_txt):
    articles.clean2_txt[i] = remove_stop_words2(articles.clean2_txt[i])
    print(i,'done')


In [ ]:
articles['clean_txt'] = articles['clean_txt'].map(remove_stop_words2)

In [ ]:
articles.tail()

In [ ]:
articles.to_csv(r'C:\Users\User\Dropbox\PC\Documents\python\14.csv' ,sep="#")

In [ ]:
joiner = lambda x : " ".join(x)

In [ ]:
articles['clean3_txt'] = articles['keywords']

In [ ]:
articles['clean3_txt'] = articles['clean2_txt'].map(joiner)

In [ ]:
articles['clean3_txt'] = articles['clean3_txt'].map(dediac_ar)

In [ ]:
articles['clean3_txt'] = articles['clean2_txt'].map(ar_stemmer)

In [ ]:
for i,txt in enumerate(articles.clean3_txt):
    articles.clean3_txt[i] = ar_stemmer(articles.clean3_txt[i])
    print(i,'done')


In [ ]:
articles['clean3_txt'] = articles['clean3_txt'].map(remove_single_double_char)

In [ ]:
articles['keywords'] = articles['clean3_txt'].map(tokenize).map(word_counter)

In [ ]:
articles['ar_keywords'] = articles['folder']

In [ ]:
for i,txt in enumerate(articles.ar_keywords):
    articles.ar_keywords[i] = only_ar_text(articles.keywords[i])
    print(i,'done')


In [ ]:
# saving models and dataframes
import joblib
# or from sklearn.externals import joblib

In [ ]:
joblib.dump(articles, "article_scope_model.pkl")

['article_scope_model.pkl']